In [ ]:

import time
from Arm_Lib import Arm_Device
from ultralytics import YOLO

# connect to DOFBOT
arm = Arm_Device()
time.sleep(0.2)

# load trained YOLO model 
model = YOLO("best.pt")

print("Arm connected.")
print("Model loaded. Classes:", model.names)


In [ ]:

# Basic arm functions

def open_gripper():
    """Open the gripper."""
    arm.Arm_serial_servo_write(6, 60, 400)
    time.sleep(0.4)


def close_gripper():
    """Close the gripper."""
    arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(0.4)


def move_arm(angles, duration=800):
    """Move servos 1–5 to the given angles.
    angles: [s1, s2, s3, s4, s5]
    duration: movement time in ms
    """
    for i in range(5):
        sid = i + 1
        val = angles[i]
        t = int(duration * 1.2) if sid == 5 else duration
        arm.Arm_serial_servo_write(sid, val, t)
        time.sleep(0.01)
    time.sleep(duration / 1000.0)


def go_up():
    """Bring the arm to a safe 'up' posture."""
    arm.Arm_serial_servo_write(2, 90, 1000)
    arm.Arm_serial_servo_write(3, 90, 1000)
    arm.Arm_serial_servo_write(4, 90, 1000)
    time.sleep(0.8)


def go_home():
    """Return to the neutral home position."""
    move_arm(home_pos, duration=900)


print("Helper functions ready.")


In [ ]:
# Arm poses 
# Neutral / rest pose
home_pos = [90, 130, 0, 0, 90]

# Above the pickup point
pick_above = [90, 80, 50, 50, 270]

# Exact pickup point (all fruits placed here)
pick_pos = [90, 53, 33, 36, 270]

# Basket positions – adjusted

basket_apple       = [136, 66, 20, 29, 270]   # based on previous green
basket_banana      = [65, 22, 64, 56, 270]    # based on previous yellow
basket_orange      = [117, 19, 66, 56, 270]   # based on previous red
basket_pear        = [44, 66, 20, 28, 270]    # based on previous blue
basket_strawberry  = [150, 60, 22, 30, 270]   # new pose – fine tune on robot
basket_watermelon  = [30, 60, 22, 30, 270]    # new pose – fine tune on robot

print("Positions configured.")


In [ ]:

# YOLO class name -> basket pose mapping

fruit_to_basket = {
    "apple":       basket_apple,
    "banana":      basket_banana,
    "orange":      basket_orange,
    "pear":        basket_pear,
    "strawberry":  basket_strawberry,
    "watermelon":  basket_watermelon,
}

def basket_for_class(name):
    key = name.strip().lower()
    if key in fruit_to_basket:
        return fruit_to_basket[key]
    print(f"[WARN] No basket defined for '{name}'.")
    return None

print("Class-to-basket mapping ready.")


In [ ]:


# Pick from common pickup point and place into basket

def pick_and_place(target_pos, label=""):
    if target_pos is None:
        return

    print(f"[INFO] Sorting: {label}")

    # ensure open at start, and go home
    open_gripper()
    go_home()

    # move to pickup
    move_arm(pick_above, duration=1000)
    move_arm(pick_pos, duration=1000)

    # grab fruit
    close_gripper()

    # lift
    move_arm(pick_above, duration=1000)

    # go to basket
    move_arm(target_pos, duration=1000)

    # release
    open_gripper()

    # back up and home
    go_up()
    go_home()

    print("[INFO] Completed one item.\n")


In [ ]:

# YOLO detection from camera

def detect_fruit_class():
    # imgsz and conf can be adjusted depending on performance
    results = model.predict(source=0, imgsz=480, conf=0.6, verbose=False)

    if not results or len(results[0].boxes) == 0:
        print("[INFO] No fruit detected.")
        return None

    # use the highest-confidence detection
    boxes = results[0].boxes
    best_idx = int(boxes.conf.argmax())
    cls_id = int(boxes.cls[best_idx])
    name = str(model.names[cls_id]).lower()

    print("[INFO] Detected class:", name)
    return name

print("Detection function ready.")


In [ ]:
# Main automatic sorting loop


try:
    print("[INFO] Starting automatic sorting loop...")

    while True:
        fruit = detect_fruit_class()
        if fruit is None:
            # nothing seen, go to next iteration
            continue

        target = basket_for_class(fruit)
        if target is None:
            # class not mapped -> skip
            continue

        pick_and_place(target_pos=target, label=fruit)

except KeyboardInterrupt:
    print("[INFO] Stopped by user.")

print("[INFO] Loop ended.")


In [ ]:

# Optional clean-up cell 

try:
    del arm
    print("Arm object deleted.")
except NameError:
    print("Arm was already deleted or not defined.")
